# Generate interval best execution benchmarks using dedicated service - F#

### Overview
Best execution aims to measure the performance of executed trades, an execution algorithm or an execution venue.

This sample is based on an `interval` approach and designed to illustrate market activity over a look back period.

<div class="alert alert-block alert-info">
    <b>Note:</b> To explore the point-in-time approach, suitable to highlight individual tradres among market activity. Please refer to <i>bestex-pointintime</i> notebooks.
</div>

This sample demonstrates how to request and plot from a dedicated data service **on-demand** best execution *interval* benchmarks.

### Inputs/outputs
Best execution interval sample requires instrument's identifier, date time intervals as per inputs. Results are as follows:
* a snapshot of occured trades
* a set of computed benchmarks based on the occured trades

### Services used
This sample uses *gRPC requests* in order to retrieve trades information from the hosted service. The queried endpoint in this script are:
* *TickTradesService*: to directly retrieve trades data from the server.

### Packages required
1. Systemathics:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source:
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package
    
***

# Run BestEx interval sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 2.38.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: CsvHelper"
#r "nuget: TaskBuilder.fs"

In [ ]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.Tick.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
// Plot charts
open XPlot.Plotly 
// Import/export csv files 
open CsvHelper
open System.IO
open System.Globalization

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request
To request *tick data* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick data request parameters

#### 3.1 Instrument selection

In [ ]:
let  ticker = "AAPL"
let  exchange = "BATS"

#### 3.2 Time period delimitation

In [ ]:
// Create the date interval (we are using Google date time format)
let  day = new DateTime(2021, 03, 15)
let  dateIntervals = 
    new DateInterval(
        StartDate = new Date ( Year = 2021, Month = 03, Day = 15 ),
        EndDate = new Date ( Year = 2021, Month = 03, Day = 15 )
    )

// Create the time interval (we are using Google date time format)
// UTC time zone
let  timeInterval = 
    new TimeInterval(
        StartTime = new TimeOfDay ( Hours = 16, Minutes = 20, Seconds = 00 ),
        EndTime = new TimeOfDay ( Hours = 16, Minutes = 30, Seconds = 00 ) 
    )

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [ ]:
// Generate constraints based on the previous time selection:
let  constraints = new Constraints() 
constraints.DateIntervals.Add(dateIntervals)
constraints.TimeIntervals.Add(timeInterval)

The following code snippets create request and instantiate the service:

In [ ]:
// Generate the tick trades request
let  request = new TickTradesRequest ( Constraints = constraints )
let  identifier = new Identifier ( Exchange = exchange, Ticker = ticker)
request.Identifiers.Add [| identifier |]

In [ ]:
// Instantiate the tick trades service
let  service = new TickTradesService.TickTradesServiceClient(channel)

### Step 4: Visualize data

#### 4.1 Retrieve trade prices and volumes

In [ ]:
// Get the trades
open FSharp.Control.Tasks.V2
open System.Collections.Generic

let  trades = new List<Trade>()

let iterAsync (t: IAsyncEnumerable<'T>) (f : 'T -> unit) =
    let task = 
        task {
            let e = t.GetAsyncEnumerator()
            let mutable go = true
            try
                let! step = e.MoveNextAsync()
                go <- step
                while go do 
                    f e.Current
                    let! step = e.MoveNextAsync()
                    go <- step
            finally
                e.DisposeAsync().AsTask().Wait()
        }
    task.Wait()

let add (collection : List<Trade>) (item : TickTradesResponse) =
    if( item |> isNull |> not && item.Trade |> isNull |> not ) then
        collection.Add item.Trade

let  call = service.TickTrades(request, headers)
let e = call.ResponseStream.ReadAllAsync()

iterAsync e (add trades)


The following code snippet displays request results:

In [ ]:
trades |> display

#### 4.2 Retrieve interval benchmarks

In [ ]:
//Get bestex time interval
//dateIntervals.StartDate.Year
let  intervalStart = new DateTime(dateIntervals.StartDate.Year, dateIntervals.StartDate.Month, dateIntervals.StartDate.Day, timeInterval.StartTime.Hours, timeInterval.StartTime.Minutes, timeInterval.StartTime.Seconds)
let  intervalEnd = new DateTime(dateIntervals.EndDate.Year, dateIntervals.EndDate.Month, dateIntervals.EndDate.Day, timeInterval.EndTime.Hours, timeInterval.EndTime.Minutes, timeInterval.EndTime.Seconds)


In [ ]:
// Get bestex time interval
let  intervalStart = new DateTime(dateIntervals.StartDate.Year, dateIntervals.StartDate.Month, dateIntervals.StartDate.Day, timeInterval.StartTime.Hours, timeInterval.StartTime.Minutes, timeInterval.StartTime.Seconds)
let  intervalEnd = new DateTime(dateIntervals.EndDate.Year, dateIntervals.EndDate.Month, dateIntervals.EndDate.Day, timeInterval.EndTime.Hours, timeInterval.EndTime.Minutes, timeInterval.EndTime.Seconds)

let getTime (t : Trade) =
    t.TimeStamp.ToDateTime()

// Get first and last trades' timestamps
let  firstTimestamp =  trades |> Seq.head |> getTime
let  lastTimestamp = trades |> Seq.last |> getTime

// Get a set of benchmarks
let  totalTrades = trades.Count
let  totalSizes = trades |> Seq.sumBy (fun trade -> trade.Size)
let  avgPrice = trades |> Seq.averageBy (fun elt -> elt.Price)
let  avgSize = trades |> Seq.averageBy (fun elt -> (double elt.Size))
let  maxPrice = trades |> Seq.maxBy (fun trade -> trade.Price)
let  minPrice = trades |> Seq.minBy (fun trade -> trade.Price)
let  maxSize = trades |> Seq.maxBy ( fun trade -> trade.Size)
let  minSize = trades |> Seq.minBy (fun trade -> trade.Size)
let  vwap = (trades |> Seq.sumBy ( fun trade -> trade.Price * (float trade.Size))) / (float totalSizes)

In [ ]:
// Display results
$"Interval start time \t: {intervalStart:s}" |> display
$"Interval end time\t: {intervalEnd:s}" |> display
$"First trade occured at\t: {firstTimestamp:s}" |> display
$"Last trade occured at\t: {lastTimestamp:s}" |> display
$"Total trades retrieved\t: {trades.Count:N0}" |> display
$"Cumulated trade sizes\t: {totalSizes:N0}" |> display
$"Average trade price\t: {avgPrice:F3}" |> display
$"Average trade size\t: {avgSize:N0}" |> display
$"Max trade price\t\t: {maxPrice:F3}" |> display
$"Min trade price\t\t: {minPrice:F3}" |> display
$"Max trade size\t\t: {maxSize:N0}" |> display
$"Min trade size\t\t: {minSize:N0}" |> display
$"Vwap\t\t\t: {vwap:F3}" |> display

#### 4.3 Plot the interval trades
Display trade prices and sizes over the look back period with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package.

In [ ]:
let  pricesAndVolumes = 
    [
        new Scattergl(
            name = "Tradeprice",
            mode = "lines",
            x = (trades |> Seq.map (fun elt -> elt.TimeStamp.ToDateTime())),
            y = (trades |> Seq.map (fun elt -> elt.Price))
        )
        new Scattergl(
            name = "Tradesize",
            x = (trades |> Seq.map (fun elt -> elt.TimeStamp.ToDateTime())),
            y = (trades |> Seq.map (fun elt -> elt.Size)),
            yaxis = "y2"
        )
    ]


let  layout = 
    new Layout.Layout( 
        title = $"Interval trades for (ticker)-(exchange) | Trades count: (totalTrades) | Toal traded volume: (totalSizes) ",
        xaxis = new Xaxis (title = "Time"),
        yaxis = new Yaxis (
            title = "Price", 
            autorange = true,
            side = "left"
        ),
        yaxis2 = new Yaxis (
            title = "Size", 
            autorange = true,
            side = "right"
        )
    )

Chart.Plot(pricesAndVolumes, layout)
|> Chart.WithWidth  1500
|> Chart.WithHeight  800
|> display